In [ ]:
import logging
import os
import sys
import shutil
import time
from pathlib import Path
from typing import Iterator, Iterable, Any
import numpy as np
import marimo as mo

# Adjust path to include src if running from root or tutorials
current_dir = Path.cwd()
if (current_dir / "src").exists():
    sys.path.append(str(current_dir / "src"))
elif (current_dir.parent / "src").exists():
    sys.path.append(str(current_dir.parent / "src"))

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s")
logger = logging.getLogger("matome.uat")

In [ ]:
mo.md(
    """
    # Matome 2.0: User Acceptance Test & Tutorial

    This notebook demonstrates the core capabilities of the Matome 2.0 "Knowledge Installation" system.
    It covers the entire pipeline from raw text to a structured, interactive knowledge base.

    **Scenarios:**
    1.  **Cycle 01: DIKW Generation**: Generate a tree where Root is Wisdom.
    2.  **Cycle 03: Semantic Zooming**: Traverse from Wisdom -> Knowledge -> Information -> Data.
    3.  **Cycle 02/04: Interactive Refinement**: Refine a node and verify persistence.
    4.  **Cycle 05: Traceability**: Verify source chunks for a summary node.
    5.  **GUI Launch**: Instructions to explore visually.

    **Modes:**
    *   **Real Mode**: Uses OpenAI/OpenRouter API for actual summarization (Requires `OPENROUTER_API_KEY`).
    *   **Mock Mode**: Uses random embeddings and dummy summaries (Default if no key found).
    """
)

# Determine Mode
api_key = os.getenv("OPENROUTER_API_KEY")
mock_mode = not bool(api_key) or api_key == "mock"

if mock_mode:
    mode_msg = "⚠️ **MOCK MODE ACTIVE** (No API Key found or set to 'mock'). Using dummy data."
else:
    mode_msg = "✅ **REAL MODE ACTIVE**. Using live API."

mo.md(mode_msg)

In [ ]:
# --- Configuration & Mocks ---
from domain_models.config import ProcessingConfig
from domain_models.manifest import Chunk, SummaryNode
from domain_models.types import DIKWLevel, NodeID
from matome.engines.embedder import EmbeddingService
from matome.agents.summarizer import SummarizationAgent
from matome.interfaces import PromptStrategy

# Initialize Config
# Ensure strict consistency for testing
config = ProcessingConfig(
    max_tokens=1000,
    max_summary_tokens=200, # Keep summaries concise
    dikw_topology={
        "root": DIKWLevel.WISDOM,
        "intermediate": DIKWLevel.KNOWLEDGE,
        "leaf": DIKWLevel.INFORMATION,
    }
)

# Mock Classes
class MockEmbeddingService(EmbeddingService):
    """Generates random embeddings for testing."""
    def __init__(self, config: ProcessingConfig):
        super().__init__(config)
        self.dim = 384  # Simulating all-MiniLM-L6-v2

    def embed_strings(self, texts: list[str] | tuple[str, ...]) -> Iterator[list[float]]:
        for _ in texts:
            # Deterministic random for stability based on length
            yield list(np.random.rand(self.dim))

    def embed_chunks(self, chunks: list[Chunk]) -> Iterator[Chunk]:
        for chunk in chunks:
            chunk.embedding = list(np.random.rand(self.dim))
            yield chunk

class MockSummarizationAgent(SummarizationAgent):
    """Generates dummy summaries respecting DIKW levels."""
    def __init__(self, config: ProcessingConfig):
        self.config = config
        self.mock_mode = True
        self.model_name = "mock-model"
        self.llm = None

    def summarize(
        self,
        text: str,
        config: ProcessingConfig | None = None,
        strategy: PromptStrategy | None = None,
        context: dict[str, Any] | None = None,
    ) -> str:
        prefix = "Summary"
        if strategy:
            # strategy.dikw_level is likely a DIKWLevel Enum member
            try:
                # Access the .value if it's an enum, or just str()
                level = getattr(strategy, "target_dikw_level", "UNKNOWN")
                # If it's an Enum, get value
                if hasattr(level, "value"):
                    level = level.value
            except AttributeError:
                level = "UNKNOWN"

            # Check context for instruction (Refinement)
            instruction = context.get("instruction") if context else None
            if instruction:
                return f"[REFINED] {instruction} -> {text[:30]}..."

            prefix = f"[{str(level).upper()}] Summary"

        return f"{prefix} of: {text[:30]}... (Mocked Content)"

# Factory
def get_services(cfg, is_mock):
    if is_mock:
        return MockEmbeddingService(cfg), MockSummarizationAgent(cfg)
    else:
        return EmbeddingService(cfg), SummarizationAgent(cfg)

mo.md("### System Configuration Loaded")

In [ ]:
# --- Step 0: Setup Test Data ---
test_data_dir = Path("test_data")
test_data_dir.mkdir(exist_ok=True)

sample_file = test_data_dir / "sample.txt"
if not sample_file.exists():
    sample_file.write_text(
        "Matome 2.0 is a system for knowledge installation. "
        "It uses recursive summarization to build a tree of knowledge. "
        "This allows users to zoom from high-level wisdom to low-level data. "
        "The system is built on Python and uses modern libraries like Pydantic and LangChain. "
        "It supports both batch processing and interactive refinement. " * 10,
        encoding="utf-8"
    )

mo.md(f"### Test Data Ready\n- `{sample_file}`")

In [ ]:
# --- Step 1: Cycle 01 - DIKW Generation ---
from matome.engines.raptor import RaptorEngine
from matome.engines.token_chunker import JapaneseTokenChunker
from matome.engines.cluster import GMMClusterer
from matome.utils.store import DiskChunkStore

mo.md("## 1. Cycle 01: DIKW Generation")

# Clean DB
store_path = Path("tutorials/chunks.db")
if store_path.exists():
    store_path.unlink()

store = DiskChunkStore(db_path=store_path)

# Setup Components
chunker = JapaneseTokenChunker(config)
clusterer = GMMClusterer()
embedder, summarizer = get_services(config, mock_mode)

engine = RaptorEngine(
    chunker=chunker,
    embedder=embedder,
    clusterer=clusterer,
    summarizer=summarizer,
    config=config
)

# Run Pipeline
text = sample_file.read_text(encoding="utf-8")
tree = engine.run(text, store=store)

# Verify Root is Wisdom
root_node = tree.root_node
# Note: In mock mode, if single chunk, it might default to WisdomStrategy manually in code
# or if recursion happened, it uses topology.

# Check DIKW level in metadata
dikw_level = root_node.metadata.dikw_level

# Assertion
assert dikw_level == DIKWLevel.WISDOM, f"Root node should be WISDOM, got {dikw_level}"

mo.md(
    f"### DIKW Generation Successful\n"
    f"Root Node ID: `{root_node.id}`\n"
    f"DIKW Level: **{dikw_level}**\n"
    f"Text Preview: {root_node.text[:100]}..."
)

In [ ]:
# --- Step 2: Cycle 03 - Semantic Zooming ---
mo.md("## 2. Cycle 03: Semantic Zooming")

# Traverse hierarchy
# Root (Wisdom) -> Children (Knowledge) -> Children (Information) -> Data (Chunks)

layers = {}

# Layer 1: Wisdom (Root)
layers[1] = [tree.root_node]

# Get children recursively
def get_children_nodes(parent_nodes):
    children = []
    for p in parent_nodes:
        # child indices can be str (SummaryNode) or int (Chunk)
        child_ids = [str(c) for c in p.children_indices]
        nodes = list(store.get_nodes(child_ids))
        children.extend([n for n in nodes if n is not None])
    return children

# Layer 2: Knowledge (Intermediate)
# Depending on text size, we might skip straight to Information if small.
# But let's see what we got.

current_layer_nodes = layers[1]
depth = 1

hierarchy_desc = []
hierarchy_desc.append(f"**Level {depth} ({current_layer_nodes[0].metadata.dikw_level})**: {len(current_layer_nodes)} node(s)")

while True:
    next_nodes = get_children_nodes(current_layer_nodes)
    if not next_nodes:
        break

    # Check if next nodes are chunks or summaries
    first_child = next_nodes[0]
    depth += 1

    # SummaryNodes have 'children_indices', Chunks do not
    if hasattr(first_child, "children_indices"): # SummaryNode
         level_name = first_child.metadata.dikw_level
         hierarchy_desc.append(f"**Level {depth} ({level_name})**: {len(next_nodes)} node(s)")
         current_layer_nodes = next_nodes
    else: # Chunk
         hierarchy_desc.append(f"**Level {depth} (DATA)**: {len(next_nodes)} chunk(s)")
         break

mo.md(f"### Hierarchy Verified\n" + "\n".join([f"- {h}" for h in hierarchy_desc]))

In [ ]:
# --- Step 3: Cycle 02/04 - Interactive Refinement ---
from matome.engines.interactive_raptor import InteractiveRaptorEngine

mo.md("## 3. Cycle 02/04: Interactive Refinement")

interactive_engine = InteractiveRaptorEngine(store, summarizer, config)

# Pick the root node to refine
target_node_id = root_node.id
instruction = "Explain like I'm 5"

# Refine
refined_node = interactive_engine.refine_node(target_node_id, instruction)

# Verify
assert refined_node.metadata.is_user_edited == True
assert instruction in refined_node.metadata.refinement_history

# Verify persistence
persisted_node = store.get_node(target_node_id)
assert persisted_node.text == refined_node.text
assert persisted_node.metadata.is_user_edited == True

mo.md(
    f"### Refinement Successful\n"
    f"Node `{target_node_id}` updated.\n"
    f"Instruction: *'{instruction}'*\n"
    f"New Text: {refined_node.text[:100]}..."
)

In [ ]:
# --- Step 4: Cycle 05 - Traceability ---
mo.md("## 4. Cycle 05: Traceability")

# Get source chunks for the root node
source_chunks = list(interactive_engine.get_source_chunks(root_node.id))

assert len(source_chunks) > 0
first_chunk = source_chunks[0]

mo.md(
    f"### Traceability Verified\n"
    f"Node `{root_node.id}` traces back to **{len(source_chunks)}** original chunks.\n"
    f"First Chunk Preview: *{first_chunk.text[:50]}...*"
)

In [ ]:
# --- Step 5: GUI Launch ---
mo.md(
    f"""
    ## 🎉 All Systems Go!

    The Matome 2.0 pipeline has been verified.
    You can now launch the interactive GUI to explore the generated knowledge base.

    Run this command in your terminal:
    ```bash
    uv run matome serve {store_path}
    ```
    """
)